For this challenge, I will be attempting to predict the arrival delay time using a regression model. 

The first thing we want to do is download the data from http://transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time. We will start with just the January 2015 data. Since the data is fairly small in size, I just took all the data. 

In [59]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')

Next, we want to eliminate the features/columns with too much missing data. Even though such features might contain useful information, there is simply not enough data on them to result in a model with accurate generalization. The threshold that we will use is 90%.

In [60]:
columnsToRemove = []
for i in range(len(df.columns)):
    numMissing = sum(pd.isnull(df.ix[:,i]))
    if (numMissing / float(len(df.index))) > 0.9:
        columnsToRemove.append(i)
namesToRemove = [df.columns.values[i] for i in columnsToRemove]
print namesToRemove  

['CANCELLATION_CODE', 'FIRST_DEP_TIME', 'TOTAL_ADD_GTIME', 'LONGEST_ADD_GTIME', 'DIV_REACHED_DEST', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE', 'DIV1_AIRPORT', 'DIV1_AIRPORT_ID', 'DIV1_AIRPORT_SEQ_ID', 'DIV1_WHEELS_ON', 'DIV1_TOTAL_GTIME', 'DIV1_LONGEST_GTIME', 'DIV1_WHEELS_OFF', 'DIV1_TAIL_NUM', 'DIV2_AIRPORT', 'DIV2_AIRPORT_ID', 'DIV2_AIRPORT_SEQ_ID', 'DIV2_WHEELS_ON', 'DIV2_TOTAL_GTIME', 'DIV2_LONGEST_GTIME', 'DIV2_WHEELS_OFF', 'DIV2_TAIL_NUM', 'DIV3_AIRPORT', 'DIV3_AIRPORT_ID', 'DIV3_AIRPORT_SEQ_ID', 'DIV3_WHEELS_ON', 'DIV3_TOTAL_GTIME', 'DIV3_LONGEST_GTIME', 'DIV3_WHEELS_OFF', 'DIV3_TAIL_NUM', 'DIV4_AIRPORT', 'DIV4_AIRPORT_ID', 'DIV4_AIRPORT_SEQ_ID', 'DIV4_WHEELS_ON', 'DIV4_TOTAL_GTIME', 'DIV4_LONGEST_GTIME', 'DIV4_WHEELS_OFF', 'DIV4_TAIL_NUM', 'DIV5_AIRPORT', 'DIV5_AIRPORT_ID', 'DIV5_AIRPORT_SEQ_ID', 'DIV5_WHEELS_ON', 'DIV5_TOTAL_GTIME', 'DIV5_LONGEST_GTIME', 'DIV5_WHEELS_OFF', 'DIV5_TAIL_NUM', 'Unnamed: 109']


Next, we remove some redundant features that are sufficiently replicated by other variables (most or all of the information in one variable is contained in the others):

In [61]:
redundantColNames = ['FL_DATE', 'CARRIER', 'ORIGIN_AIRPORT_SEQ_ID', 
                     'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
                     'ORIGIN_STATE_NM', 'ORIGIN_WAC', 'DEST_AIRPORT_SEQ_ID',
                     'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_FIPS',
                     'DEST_STATE_NM', 'DEST_WAC', 'CRS_DEP_TIME', 'DEP_TIME',
                     'DEP_DEL15', 'DEP_DELAY_GROUP', 'CRS_ARR_TIME', 
                     'ARR_TIME', 'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP',
                     'DISTANCE_GROUP', 'YEAR', 'QUARTER', 'MONTH', 'FLIGHTS']
namesToRemove.extend(redundantColNames)

One thing to note is that ARR_DELAY_NEW was removed as a potential output variable and ARR_DELAY will be used instead. For regression, it may be the case that using ARR_DELAY_NEW would actually yield better performance, but I did not test this due to time constraints.

In order for our model to be useful, it should only incorporate features that can be known before the flight happens. It should not, for example, include features in the data such as departure delay, weather delay, or wheels off time, otherwise that defeats the point of the prediction. Therefore, we remove all such features:

In [62]:
nonIndependentColNames = ['DEP_DELAY', 'DEP_DELAY_NEW', 'WHEELS_OFF', 
                          'WHEELS_ON', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME',
                          'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
                          'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
namesToRemove.extend(nonIndependentColNames)

Now we look at the features of our reduced dataset:

In [63]:
for colName in namesToRemove:
    df.drop(colName, axis=1, inplace=True)

delays = df['ARR_DELAY']
df.drop('ARR_DELAY', axis=1, inplace=True)
print (df.columns.values)

['DAY_OF_MONTH' 'DAY_OF_WEEK' 'UNIQUE_CARRIER' 'AIRLINE_ID' 'TAIL_NUM'
 'FL_NUM' 'ORIGIN_AIRPORT_ID' 'ORIGIN_CITY_MARKET_ID' 'ORIGIN'
 'DEST_AIRPORT_ID' 'DEST_CITY_MARKET_ID' 'DEST' 'DEP_TIME_BLK' 'TAXI_OUT'
 'TAXI_IN' 'ARR_TIME_BLK' 'CANCELLED' 'DIVERTED' 'CRS_ELAPSED_TIME'
 'DISTANCE' 'DIV_AIRPORT_LANDINGS']


Now we can build a baseline model for the data. Since we've kept many of the variables as discrete variables, we cannot directly apply models such as linear regression models or variants like elastic net. Elastic net models have proven to be quite versatile in many situations, and we can implement it by converting the discrete variables to binary indicator variables (one for each level in the variable). However, for now, I will implement models that can handle discrete variables. 

In the past, there has been work done on the topic of flight delay predictions. In particular, the winners of a crowdsourcing competition, Flight Quest 1 (http://www.gequest.com/c/flight), used a combination of gradient boosting  and random forest models. Therefore, we will implement these 2 baseline models.

For GBM, we set max_features to a default value equal to the number of features, or 21.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

discrete_features = ['DAY_OF_MONTH', 'DAY_OF_WEEK', 'UNIQUE_CARRIER', 
                     'AIRLINE_ID', 'TAIL_NUM', 'FL_NUM', 'ORIGIN_AIRPORT_ID', 
                     'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 
                     'DEST_CITY_MARKET_ID', 'DEST', 'DEP_TIME_BLK', 'ARR_TIME_BLK',
                     'CANCELLED', 'DIVERTED']
discrete_indices = [for i in range(len(df.columns.values))]



In [37]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation

X = np.array(df)
y = delays.values
gbm_model = GradientBoostingRegressor(n_estimators=100, max_features=21)
cross_val_results = cross_validation.cross_val_score(gbm_model, X, y, cv=10, scoring='mean_squared_error')
print cross_val_results

ValueError: invalid literal for float(): 0800-0859

In [53]:
from sklearn.ensemble import RandomForestRegressor

X = np.array(df)
y = delays.values
print y
rf_model = RandomForestRegressor(n_estimators=100, max_features=21)
cross_val_results = cross_validation.cross_val_score(rf_model, X, y, cv=10, scoring='mean_squared_error')
print cross_val_results

[-16.  31. -14. ...,   2.  13.  29.]


ValueError: invalid literal for float(): 0800-0859

This model is simplistic in that it assumes that airports delays have not changed over time, which is not necessarily true. One way to rectify would be to apply weights to each observation, with recent flights being weighted more heavily. Might be useful to encode an isHoliday variable?

In [58]:
print df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469968 entries, 0 to 469967
Data columns (total 21 columns):
DAY_OF_MONTH             469968 non-null category
DAY_OF_WEEK              469968 non-null category
UNIQUE_CARRIER           469968 non-null category
AIRLINE_ID               469968 non-null category
TAIL_NUM                 467186 non-null category
FL_NUM                   469968 non-null category
ORIGIN_AIRPORT_ID        469968 non-null category
ORIGIN_CITY_MARKET_ID    469968 non-null category
ORIGIN                   469968 non-null category
DEST_AIRPORT_ID          469968 non-null category
DEST_CITY_MARKET_ID      469968 non-null category
DEST                     469968 non-null category
DEP_TIME_BLK             469968 non-null category
TAXI_OUT                 458092 non-null float64
TAXI_IN                  457697 non-null float64
ARR_TIME_BLK             469968 non-null category
CANCELLED                469968 non-null category
DIVERTED                 469968 non-null 